In [1]:
import pandas as pd

# ---------- SETTINGS ----------
# Zet hier de bestandsnamen van je CSV's
MENTOREN_FILE = "../data/mentoren.csv"
STUDENTEN_FILE = "../data/studenten.csv"
OUTPUT_FILE = "../output/matches.csv"

# ---------- DATA INLADEN ----------
mentoren_df = pd.read_csv(MENTOREN_FILE)
studenten_df = pd.read_csv(STUDENTEN_FILE)

# Zorg dat onderwerpen correct als lijst worden gelezen
mentoren_df["Onderwerpen"] = mentoren_df["Onderwerpen"].apply(
    lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x]
)

# Opleidingsniveaus ordenen
level_order = {"Associate": 0, "Bachelor": 1, "Master": 2, "PhD": 3}

# ---------- MATCHING FUNCTIE ----------
def find_best_mentor(student, mentoren_df):
    student_level = level_order.get(student["Opleidingsniveau"], -1)
    student_subject = student["Onderwerp"]

    best_match = None
    best_score = -1

    for _, mentor in mentoren_df.iterrows():
        mentor_level = level_order.get(mentor["Opleidingsniveau"], -1)

        # Mentor moet hoger niveau hebben
        if mentor_level <= student_level:
            continue

        # Onderwerp match
        subject_match = 1 if student_subject in mentor["Onderwerpen"] else 0

        # Score = niveauverschil + extra punten voor onderwerp
        score = (mentor_level - student_level) + (subject_match * 2)

        if score > best_score:
            best_score = score
            best_match = mentor

    if best_match is not None:
        return {
            "Student": f"{student['Voornaam']} {student['Achternaam']}",
            "Student_niveau": student["Opleidingsniveau"],
            "Onderwerp": student_subject,
            "Mentor": f"{best_match['Voornaam']} {best_match['Achternaam']}",
            "Mentor_niveau": best_match["Opleidingsniveau"],
            "Mentor_onderwerpen": best_match["Onderwerpen"],
            "Match_score": best_score,
        }
    else:
        return {
            "Student": f"{student['Voornaam']} {student['Achternaam']}",
            "Student_niveau": student["Opleidingsniveau"],
            "Onderwerp": student_subject,
            "Mentor": None,
            "Mentor_niveau": None,
            "Mentor_onderwerpen": None,
            "Match_score": 0,
        }

# ---------- MATCHES MAKEN ----------
matches = [find_best_mentor(student, mentoren_df) for _, student in studenten_df.iterrows()]
matches_df = pd.DataFrame(matches)

# ---------- RESULTAAT OPSLAAN ----------
matches_df.to_csv(OUTPUT_FILE, index=False)
print(f"Matching voltooid! Resultaat opgeslagen in {OUTPUT_FILE}")

Matching voltooid! Resultaat opgeslagen in ../output/matches.csv


In [2]:
import random
import pandas as pd


class Student:
    def __init__(self, naam: str, niveau: str, onderwerp: str):
        self.naam = naam
        self.niveau = niveau
        self.onderwerp = onderwerp

    def __repr__(self):
        return f"Student(naam={self.naam}, niveau={self.niveau}, onderwerp={self.onderwerp})"

class Mentor:
    def __init__(self, naam: str, niveau: str, onderwerpen: list[str], max_studenten: int):
        self.naam = naam
        self.niveau = niveau
        self.onderwerpen = onderwerpen
        self.max_studenten = max_studenten

    def __repr__(self):
        return f"Mentor(naam={self.naam}, niveau={self.niveau}, onderwerpen={self.onderwerpen}, max_studenten={self.max_studenten})"
    

def vul_studenten(studenten_lijst, csv_path, n):
    df = pd.read_csv(csv_path)
    rows = df.sample(n=min(n, len(df)))
    for _, row in rows.iterrows():
        naam = f"{row['Voornaam']} {row['Achternaam']}" if 'Voornaam' in row and 'Achternaam' in row else row.get('Naam', '')
        studenten_lijst.append(Student(naam, row['Opleidingsniveau'], row['Onderwerp']))


def vul_mentoren(mentoren_lijst, csv_path, n):
    df = pd.read_csv(csv_path)
    # Zorg dat onderwerpen als lijst worden gelezen
    df["Onderwerpen"] = df["Onderwerpen"].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    rows = df.sample(n=min(n, len(df)))
    for _, row in rows.iterrows():
        naam = f"{row['Voornaam']} {row['Achternaam']}" if 'Voornaam' in row and 'Achternaam' in row else row.get('Naam', '')
        max_studenten = random.randint(1, 5)
        mentoren_lijst.append(Mentor(naam, row['Opleidingsniveau'], row['Onderwerpen'], max_studenten))

level_order = {"Associate": 0, "Bachelor": 1, "Master": 2, "PhD": 3}

studenten = []
mentoren = []

vul_studenten(studenten, "../data/studenten.csv", 90)
vul_mentoren(mentoren, "../data/mentoren.csv", 30)

print(studenten)
print(mentoren)




[Student(naam=Joris van Asten, niveau=Associate, onderwerp=Ethical Hacking), Student(naam=Joost Rutten, niveau=Bachelor, onderwerp=Artificial Intelligence), Student(naam=Aylin Galijn, niveau=Bachelor, onderwerp=Creative Digital Innovation), Student(naam=Micha Claesdr, niveau=Bachelor, onderwerp=Creative Digital Innovation), Student(naam=Thijmen Saxo, niveau=Associate, onderwerp=Artificial Intelligence), Student(naam=Dewi Houdijk, niveau=Bachelor, onderwerp=Ethical Hacking), Student(naam=Niek Tillmanno, niveau=Associate, onderwerp=Software Architecture), Student(naam=Arthur Jacobi, niveau=Associate, onderwerp=Creative Digital Innovation), Student(naam=Isabelle van de Elzas, niveau=Bachelor, onderwerp=Design Science Research), Student(naam=Lois Remmers, niveau=Associate, onderwerp=Design Science Research), Student(naam=Amin de Lange, niveau=Associate, onderwerp=Business Process Analytics), Student(naam=Puk ter Waarbeek, niveau=Associate, onderwerp=Business Process Analytics), Student(naa

# Uninformed 
bfs (werkt maar alleen voor kleine aantallen)

In [3]:
from collections import deque
import copy

def bfs_student_mentor_matching(studenten, mentoren):
    mentor_slots = {i: mentoren[i].max_studenten for i in range(len(mentoren))}
    initial_state = (0, [], mentor_slots)
    queue = deque([initial_state])

    while queue:
        student_idx, assignment, slots = queue.popleft()
        if student_idx == len(studenten):
            return assignment

        student = studenten[student_idx]
        assigned = False
        for mentor_idx, mentor in enumerate(mentoren):
            if (
                slots[mentor_idx] > 0 
                and student.onderwerp in mentor.onderwerpen
                and level_order[mentor.niveau] > level_order[student.niveau]
            ):
                new_slots = copy.deepcopy(slots)
                new_slots[mentor_idx] -= 1
                queue.append((student_idx + 1, assignment + [mentor_idx], new_slots))
                assigned = True
        if not assigned:
            queue.append((student_idx + 1, assignment + [None], slots))
    return None  

# result = bfs_student_mentor_matching(studenten, mentoren)
# matched_count = 0
# for idx, mentor_idx in enumerate(result):
#     student = studenten[idx]
#     if mentor_idx is not None:
#         mentor = mentoren[mentor_idx]
#         matched_count += 1
#         print(f"{student.naam} -> {mentor.naam}")
#     else:
#         print(f"{student.naam} -> GEEN MATCH")

# print(f"bfs matched: {matched_count}/{len(studenten)}")

In [4]:
import copy

def dfs_student_mentor_matching(studenten, mentoren):
    mentor_slots = {i: mentoren[i].max_studenten for i in range(len(mentoren))}
    stack = [(0, [], mentor_slots)]

    while stack:
        student_idx, assignment, slots = stack.pop()
        if student_idx == len(studenten):
            return assignment

        student = studenten[student_idx]
        assigned = False
        for mentor_idx, mentor in enumerate(mentoren):
            if (
                slots[mentor_idx] > 0
                and student.onderwerp in mentor.onderwerpen
                and level_order[mentor.niveau] > level_order[student.niveau]
            ):
                new_slots = copy.deepcopy(slots)
                new_slots[mentor_idx] -= 1
                stack.append((student_idx + 1, assignment + [mentor_idx], new_slots))
                assigned = True
        if not assigned:
            stack.append((student_idx + 1, assignment + [None], slots))
    return None  

result = dfs_student_mentor_matching(studenten, mentoren)
matched_count = 0
for idx, mentor_idx in enumerate(result):
    student = studenten[idx]
    if mentor_idx is not None:
        mentor = mentoren[mentor_idx]
        matched_count += 1
        print(f"{student.naam} -> {mentor.naam}")
    else:
        print(f"{student.naam} -> GEEN MATCH")

print(f"dfs matched: {matched_count}/{len(studenten)}")

Joris van Asten -> Joëlle Bresé
Joost Rutten -> Bart van Haspengouw
Aylin Galijn -> Isa Maas
Micha Claesdr -> Noah Niermann
Thijmen Saxo -> Joëlle Bresé
Dewi Houdijk -> Bart van Haspengouw
Niek Tillmanno -> Vigo van 't Wel
Arthur Jacobi -> Joëlle Bresé
Isabelle van de Elzas -> Noah Niermann
Lois Remmers -> Lucy Hulskes
Amin de Lange -> Marijn Bertho
Puk ter Waarbeek -> Marijn Bertho
Mason van der Horst -> Joëlle Bresé
Sep Molen -> Veerle Aertsz
Elif van Berkel -> Bart van Haspengouw
Esther Scholten -> Veerle Aertsz
Mart Kuijpers -> Bart van Haspengouw
Emir Jorrisen -> Adriana Schelvis
Rosalie Segerszoen -> Veerle Aertsz
Kyra Broeckx -> Veerle Aertsz
Isis Verbeek -> Mick Verhoeven
Arie Charon -> Linde Hollander
Daan Moenen -> Dewi Langevoort
Noortje Kuijpers -> Hidde Rehorst
Tyler Miltenburg -> Hidde Rehorst
Dion van den Corput -> Dewi Langevoort
Lisa de Swart -> Olivier Sarneel
Aaron van de Wal -> Sjoerd Carnotte
Ravi Cosman -> Chloé van Oostendorp
Kate Demmendaal -> Chloé van Oostendo

In [5]:
import heapq
import copy
import itertools

def is_compatible(student, mentor):
    """Binary compatibility check between student and mentor"""
    return (student.onderwerp in mentor.onderwerpen and 
            level_order[mentor.niveau] > level_order[student.niveau])

def heuristic(remaining_students, available_mentors, mentor_slots):
    """Optimistic estimate of remaining matching potential"""
    if not remaining_students:
        return 0
    
    matchable_count = 0
    for student in remaining_students:
        for mentor_idx, mentor in enumerate(available_mentors):
            if mentor_slots[mentor_idx] > 0 and is_compatible(student, mentor):
                matchable_count += 1
                break  
    return matchable_count



In [6]:
def greedy_best_first_matching(studenten, mentoren):
    """Greedy best-first search - picks first compatible mentor"""
    mentor_slots = {i: mentoren[i].max_studenten for i in range(len(mentoren))}
    assignment = []
    
    for student in studenten:
        mentor_assigned = None
        
        for mentor_idx, mentor in enumerate(mentoren):
            if mentor_slots[mentor_idx] > 0 and is_compatible(student, mentor):
                mentor_slots[mentor_idx] -= 1
                mentor_assigned = mentor_idx
                break  
        
        assignment.append(mentor_assigned)
    
    return assignment


print("\n=== Greedy Best-First Results ===")
greedy_result = greedy_best_first_matching(studenten, mentoren)
matched_count = 0
for idx, mentor_idx in enumerate(greedy_result):
    student = studenten[idx]
    if mentor_idx is not None:
        mentor = mentoren[mentor_idx]
        matched_count += 1
        print(f"{student.naam} -> {mentor.naam}")
    else:
        print(f"{student.naam} -> GEEN MATCH")

print(f"Greedy matched: {matched_count}/{len(studenten)}")


=== Greedy Best-First Results ===
Joris van Asten -> Jill Jans
Joost Rutten -> Lot Bave
Aylin Galijn -> Jill Jans
Micha Claesdr -> Jill Jans
Thijmen Saxo -> Amélie Kort
Dewi Houdijk -> Stella Wolfswinkel
Niek Tillmanno -> Amélie Kort
Arthur Jacobi -> Gijs van Kuijc
Isabelle van de Elzas -> Chloé van Oostendorp
Lois Remmers -> Chloé van Oostendorp
Amin de Lange -> Xavi van Wickerode
Puk ter Waarbeek -> Xavi van Wickerode
Mason van der Horst -> Thijs van Waas
Sep Molen -> Thijs van Waas
Elif van Berkel -> Renske Jonker
Esther Scholten -> Renske Jonker
Mart Kuijpers -> Stella Wolfswinkel
Emir Jorrisen -> Matthijs Verbeek
Rosalie Segerszoen -> Stella Wolfswinkel
Kyra Broeckx -> Xavi van Wickerode
Isis Verbeek -> Stella Wolfswinkel
Arie Charon -> Hidde Rehorst
Daan Moenen -> Olivier Sarneel
Noortje Kuijpers -> Hidde Rehorst
Tyler Miltenburg -> Hidde Rehorst
Dion van den Corput -> Olivier Sarneel
Lisa de Swart -> Hidde Rehorst
Aaron van de Wal -> Mats de Vroege
Ravi Cosman -> Mats de Vroege